In [61]:
%pip install stable-baselines3 gymnasium pandas matplotlib shimmy torch sb3-contrib

Note: you may need to restart the kernel to use updated packages.


In [62]:
import gymnasium as gym
import numpy as np
import torch as th
import pandas as pd
import matplotlib.pyplot as plt
import os
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.monitor import Monitor
from typing import Callable, Union, Tuple, Optional
import torch.nn as nn
from copy import deepcopy
from gymnasium import spaces
from collections import namedtuple
from enum import Enum

In [ ]:
Ship = namedtuple('Ship', ['min_x', 'max_x', 'min_y', 'max_y'])
Action = namedtuple('Action', ['x', 'y'])

class CHANNEL_MAP(Enum):
    MISSED = 0 
    HIT = 1 
    LEGAL_MOVE = 2 

class BattleshipEnv(gym.Env):
    REWARD_KEYS = ['win', 'missed', 'hit', 'proximal_hit', 'sunk_ship_bonus', 'step_penalty']

    def __init__(self,
                 board_size: Tuple = None,
                 ship_sizes: dict = None,
                 episode_steps: int = 300,
                 reward_dictionary: Optional[dict] = None):

        super().__init__()
        self.ship_sizes = ship_sizes or {5: 1, 4: 1, 3: 2, 2: 1}
        self.board_size = board_size or (10, 10)
        self.board = None 
        self.board_generated = None 
        self.observation = None 
        self.NUM_CHANNELS = 3
        self.step_count = None
        self.episode_steps = episode_steps

        default_reward_dictionary = {  
            'win': 500,
            'missed': 0,
            'hit': 5,
            'proximal_hit': 20,
            'sunk_ship_bonus': 10.0,
            'step_penalty': -0.5 
        }
        
        user_rewards = reward_dictionary or {}
        self.reward_dictionary = {key: user_rewards.get(key, default_reward_dictionary.get(key, 0)) 
                                  for key in self.REWARD_KEYS}
                                  
        self.action_space = spaces.Discrete(self.board_size[0] * self.board_size[1])
        # Observation: Channel 0=Miss, 1=Hit, 2=Legal(unused by CNN but good for viz)
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.NUM_CHANNELS, self.board_size[0], self.board_size[1]), dtype=np.float32)
    
    def _in_bounds(self, x: int, y: int) -> bool:
        return 0 <= x < self.board_size[0] and 0 <= y < self.board_size[1]
    
    def _check_proximal_hit(self, action: Action) -> bool:
        x, y = action.x, action.y
        neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
        for nx, ny in neighbors:
            if self._in_bounds(nx, ny) and self.observation[CHANNEL_MAP.HIT.value, nx, ny] == 1:
                return True
        return False
    
    # --- NEW: Action Masking Method ---
    def action_masks(self) -> np.ndarray:
        #get current hits and misses from observation
        # Channel 0 is Missed, Channel 1 is Hit
        missed_mask = self.observation[CHANNEL_MAP.MISSED.value].flatten().astype(bool)
        hit_mask = self.observation[CHANNEL_MAP.HIT.value].flatten().astype(bool)

        occupied = missed_mask | hit_mask
        
        return ~occupied

    def step(self, raw_action: Union[int, tuple]) -> Tuple[np.ndarray, float, bool, bool, dict]:
        if isinstance(raw_action, (int, np.int64)):
            action = Action(x=raw_action // self.board_size[1], y=raw_action % self.board_size[1])
        elif isinstance(raw_action, tuple):
            action = Action(x=raw_action[0], y=raw_action[1])
        else:
            raise AssertionError("Invalid raw_action type")

        self.step_count += 1
        reward = self.reward_dictionary['step_penalty']
        
        terminated = False
        truncated = self.step_count >= self.episode_steps
        
        if truncated:
            return self.observation, reward, terminated, truncated, {}
        
        x, y = action.x, action.y

       
        if self.board[x, y] > 0: 
            ship_id = self.board[x, y]
            
            if self._check_proximal_hit(action):
                reward += self.reward_dictionary['proximal_hit']
            else:
                reward += self.reward_dictionary['hit']
            
            self.board[x, y] = 0
            self.observation[CHANNEL_MAP.HIT.value, x, y] = 1
            self.observation[CHANNEL_MAP.LEGAL_MOVE.value, x, y] = 0 
            
            if not np.any(self.board == ship_id):
                reward += self.reward_dictionary['sunk_ship_bonus']
                
            if not self.board.any():
                terminated = True
                moves_taken = self.step_count
                efficiency_bonus = max(0, 300 - (moves_taken * 2))
                reward += self.reward_dictionary['win'] + efficiency_bonus
            
            return self.observation, reward, terminated, truncated, {}

        else:
            self.observation[CHANNEL_MAP.MISSED.value, x, y] = 1
            self.observation[CHANNEL_MAP.LEGAL_MOVE.value, x, y] = 0
            reward += self.reward_dictionary['missed']
            return self.observation, reward, terminated, truncated, {}

    def reset(self, seed=None, options=None) -> Tuple[np.ndarray, dict]:
        super().reset(seed=seed)
        self._set_board()
        self.board_generated = deepcopy(self.board)
        self.observation = np.zeros((self.NUM_CHANNELS, *self.board_size), dtype=np.float32)
        #initialize Legal Move to all 1s since all moves valid at start
        self.observation[CHANNEL_MAP.LEGAL_MOVE.value] = 1.0 
        self.step_count = 0
        return self.observation, {}

    def _set_board(self) -> None:
        self.board = np.zeros(self.board_size, dtype=np.int32)
        ship_id_counter = 1
        for ship_size, ship_count in self.ship_sizes.items():
            for _ in range(ship_count):
                self._place_ship(ship_size, ship_id_counter)
                ship_id_counter += 1

    def _place_ship(self, ship_size: int, ship_id: int) -> None:
        can_place_ship = False
        attempts = 0
        max_attempts = 100 
        while not can_place_ship and attempts < max_attempts:
            ship = self._get_ship(ship_size, self.board_size)
            can_place_ship = self._is_place_empty(ship)
            attempts += 1
        if can_place_ship:
            self.board[ship.min_x:ship.max_x, ship.min_y:ship.max_y] = ship_id

    @staticmethod
    def _get_ship(ship_size: int, board_size: tuple) -> Ship:
        if np.random.choice(('Horizontal', 'Vertical')) == 'Horizontal':
            min_x = np.random.randint(0, board_size[0] + 1 - ship_size)
            min_y = np.random.randint(0, board_size[1])
            return Ship(min_x=min_x, max_x=min_x + ship_size, min_y=min_y, max_y=min_y + 1)
        else:
            min_x = np.random.randint(0, board_size[0])
            min_y = np.random.randint(0, board_size[1] + 1 - ship_size)
            return Ship(min_x=min_x, max_x=min_x + 1, min_y=min_y, max_y=min_y + ship_size)

    def _is_place_empty(self, ship: Ship) -> bool:
        return np.count_nonzero(self.board[ship.min_x:ship.max_x, ship.min_y:ship.max_y]) == 0

In [68]:
class BattleshipCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super().__init__(observation_space, features_dim)
        n_channels = observation_space.shape[0] 
        self.cnn = nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )
        with th.no_grad():
            obs_tensor = th.as_tensor(observation_space.sample()[None]).float()
            conv_output_dim = self.cnn(obs_tensor).shape[1]
        self.linear = nn.Sequential(
            nn.Linear(conv_output_dim, features_dim),
            nn.ReLU()
        )
    def forward(self, obs: th.Tensor) -> th.Tensor:
        features = self.linear(self.cnn(obs.float()))
        return th.nn.functional.layer_norm(features, features.shape)

In [69]:
class CustomSaveCallback(BaseCallback):
    def __init__(self, check_freq: int, save_path: str, verbose: int = 1):
        super(CustomSaveCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        os.makedirs(self.save_path, exist_ok=True)
        self.best_mean_reward = -np.inf
    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            try:
                x, y = ts2xy(load_results(self.save_path), 'timesteps')
            except Exception: return True 
            if len(x) > 0:
                mean_reward = np.mean(y[-min(100, len(y)):])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}, Mean reward: {mean_reward:.2f}")
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    self.model.save(os.path.join(self.save_path, "best_model"))
        return True

def linear_schedule(initial_value: float) -> Callable[[float], float]:
    def func(progress_remaining: float) -> float:
        return progress_remaining * initial_value
    return func

In [ ]:
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker

reward_dict = {
    'win': 200,
    'missed': 0,
    'hit': 5,
    'proximal_hit': 20,
    'sunk_ship_bonus': 10.0,
    'step_penalty': -0.5 
}

LOG_DIR = "./ppo_battleship_v7_log/"
MODEL_PATH = os.path.join(LOG_DIR, "ppo_battleship_v7_final.zip")
CHECKPOINT_FREQ = 10000  
TOTAL_TIMESTEPS = 3000000 

def mask_fn(env: gym.Env) -> np.ndarray:
    return env.action_masks()

def make_masked_env():
    env = BattleshipEnv(reward_dictionary=reward_dict, episode_steps=300)
    #apply ActionMasker wrapper here
    env = ActionMasker(env, mask_fn)
    return env

#create Vectorized Environment
num_envs = 16
env = make_vec_env(
    make_masked_env, 
    n_envs=num_envs, 
    seed=0, 
    monitor_dir=LOG_DIR
)

policy_kwargs = dict(
    features_extractor_class=BattleshipCNN,
    features_extractor_kwargs=dict(features_dim=256),
    net_arch=dict(pi=[256, 128], vf=[256, 128]),
    normalize_images=False
)

model = MaskablePPO(
    "CnnPolicy",
    env,
    learning_rate=linear_schedule(0.0003), 
    n_steps=1024,           
    batch_size=512,            
    n_epochs=10,
    gamma=0.99,
    clip_range=0.1,
    ent_coef=0.01,
    verbose=1, 
    policy_kwargs=policy_kwargs,
    tensorboard_log=LOG_DIR
)

callback = CustomSaveCallback(check_freq=CHECKPOINT_FREQ, save_path=LOG_DIR)

print(f"Starting training for {TOTAL_TIMESTEPS} timesteps...")
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=callback, progress_bar=True)
model.save(MODEL_PATH)
print(f"Training complete.")

Using cpu device

Starting MaskablePPO training (v7) for 3000000 timesteps...

Logging to ./ppo_battleship_v7_log/PPO_4

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.2     |
|    ep_rew_mean     | 551      |
| time/              |          |
|    fps             | 1961     |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 16384    |
---------------------------------

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 95.4         |
|    ep_rew_mean          | 550          |
| time/                   |              |
|    fps                  | 622          |
|    iterations           | 2            |
|    time_elapsed         | 52           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0025956067 |
|    clip_fraction        | 0.043        |
|    clip_range           | 0.1          |
|    entropy_loss         | -3.77        |
|    explained_variance   | -0.000592    |
|    learning_rate        | 0.000298     |
|    loss                 | 5.11e+03     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00301     |
|    value_loss           | 1.21e+04     |
------------------------------------------

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 93.6         |
|    ep_rew_mean          | 560          |
| time/                   |              |
|    fps                  | 278          |
|    iterations           | 115          |
|    time_elapsed         | 6765         |
|    total_timesteps      | 1884160      |
| train/                  |              |
|    approx_kl            | 0.0035271728 |
|    clip_fraction        | 0.183        |
|    clip_range           | 0.1          |
|    entropy_loss         | -3.57        |
|    explained_variance   | 0.801        |
|    learning_rate        | 0.000113     |
|    loss                 | 3.01         |
|    n_updates            | 1140         |
|    policy_gradient_loss | 0.000553     |
|    value_loss           | 25.4         |
------------------------------------------

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 93         |
|    ep_rew_mean          | 556        |
| time/                   |            |
|    fps                  | 275        |
|    iterations           | 116        |
|    time_elapsed         | 6890       |
|    total_timesteps      | 1900544    |
| train/                  |            |
|    approx_kl            | 0.00375266 |
|    clip_fraction        | 0.169      |
|    clip_range           | 0.1        |
|    entropy_loss         | -3.54      |
|    explained_variance   | 0.834      |
|    learning_rate        | 0.000112   |
|    loss                 | 3.1        |
|    n_updates            | 1150       |
|    policy_gradient_loss | -0.00391   |
|    value_loss           | 19.6       |
----------------------------------------

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 92.1         |
|    ep_rew_mean          | 557          |
| time/                   |              |
|    fps                  | 271          |
|    iterations           | 117          |
|    time_elapsed         | 7065         |
|    total_timesteps      | 1916928      |
| train/                  |              |
|    approx_kl            | 0.0032843058 |
|    clip_fraction        | 0.152        |
|    clip_range           | 0.1          |
|    entropy_loss         | -3.55        |
|    explained_variance   | 0.864        |
|    learning_rate        | 0.00011      |
|    loss                 | 2.89         |
|    n_updates            | 1160         |
|    policy_gradient_loss | -0.00278     |
|    value_loss           | 22           |
------------------------------------------

   0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10,064/3,000,000  [ 2:13:12 < -:--:-- , ? it/s ]

KeyboardInterrupt: 

In [73]:
# Save the current model state
CURRENT_MODEL_PATH = os.path.join(LOG_DIR, "ppo_battleship_v7_stopped_at_1.9M.zip")
model.save(CURRENT_MODEL_PATH)
print(f"Model saved to {CURRENT_MODEL_PATH}")

Model saved to ./ppo_battleship_v7_log/ppo_battleship_v7_stopped_at_1.9M.zip


In [ ]:

BEST_MODEL_PATH = os.path.join(LOG_DIR, "best_model.zip")
if os.path.exists(BEST_MODEL_PATH):
    eval_model = MaskablePPO.load(BEST_MODEL_PATH)
    print("Loaded Best Model.")
else:
    eval_model = model
    print("Using Current Model.")

EVAL_EPISODES = 100

eval_env = BattleshipEnv(reward_dictionary=reward_dict, episode_steps=300)

total_moves = []
win_count = 0

print(f"\n--- Starting Evaluation of {EVAL_EPISODES} Games (v7 Masked) ---")

for episode in range(EVAL_EPISODES):
    obs, info = eval_env.reset()
    terminated, truncated = False, False
    episode_moves = 0
    
    while not (terminated or truncated):
        action_masks = eval_env.action_masks()
        
        #pass the mask to the model!
        action, _ = eval_model.predict(obs, action_masks=action_masks, deterministic=True)
        
        obs, reward, terminated, truncated, info = eval_env.step(action.item())
        episode_moves += 1

    if not eval_env.board.any(): 
        win_count += 1
        total_moves.append(episode_moves)
    
    if (episode + 1) % 10 == 0:
        print(f"Completed {episode + 1}/{EVAL_EPISODES} episodes.")

eval_env.close()

if total_moves:
    avg_moves = np.mean(total_moves)
    std_moves = np.std(total_moves)
    stats = f"**{avg_moves:.2f} +/- {std_moves:.2f}**"
else: stats = "N/A"

print("\n--- Evaluation Summary (v7) ---")
print(f"Win Rate: **{win_count / EVAL_EPISODES * 100:.2f}%**")
print(f"Avg Moves: {stats}")

Loaded Best Model.

--- Starting Evaluation of 100 Games (v7 Masked) ---
Completed 10/100 episodes.
Completed 20/100 episodes.
Completed 30/100 episodes.
Completed 40/100 episodes.
Completed 50/100 episodes.
Completed 60/100 episodes.
Completed 70/100 episodes.
Completed 80/100 episodes.
Completed 90/100 episodes.
Completed 100/100 episodes.

--- Evaluation Summary (v7) ---
Win Rate: **100.00%**
Avg Moves: **91.86 +/- 6.36**
